## Generating CLIWOC missing code tables

The Climatological Database for the World's Oceans 1750-1850 ([CLIWOC](https://stvno.github.io/page/cliwoc/)) has valuable information on its supplemental data stored in the [IMMA](https://icoads.noaa.gov/e-doc/imma/R3.0-imma1.pdf) format under the C99 column.

We have successfully extracted this information with the [mdf_reader()](https://git.noc.ac.uk/brecinosrivas/mdf_reader) tool, but several important variables are missing their code tables.

List of variables:

- Ship types
- latitude indicator
- longitude indicator,
- air temperature units
- sst units
- air pressure units
- units of attached thermometer
- longitude units
- Barometer type
- Distance units
- Distance units to land marks
- Distance units of how much the ship traveled
- Units of other measurements (e.g. current speed)
- Humidity units

According to the [documentation](https://stvno.github.io/page/cliwoc/) of this deck (730) there are up to 20 different ways of writing down the air pressure but the code tables are not available anymore on the website. Therefore, we extracted from the supplemental data all possible entries for those fields which are missing a code table. We count each entry in order to construct a code table for that particular variable.

The code to extract multiple variables from the CLIWOC supplemental data can be found [here](https://git.noc.ac.uk/brecinosrivas/mdf_reader/-/blob/master/tests/gather_stats_c99.py)

### Set up

In [ ]:
from __future__ import annotations

import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# PARAMS for plots
from matplotlib import rcParams

sns.set_style("whitegrid")
rcParams["axes.labelsize"] = 14
rcParams["xtick.labelsize"] = 14
rcParams["ytick.labelsize"] = 14
rcParams["legend.fontsize"] = 16
rcParams["legend.title_fontsize"] = 16

We stored the statistics per year in python pickle dictionaries.

In [ ]:
# Paths to data
dirs = "/Users/brivas/c3s_work/mdf_reader/tests/data/133-730/133-730"
file_names = sorted(os.listdir(dirs))

In [ ]:
file_names[0:5]

In [ ]:
def get_values(dic, key, year):
    """
    Get individual sets of values from the pickle df
    Params:
    ------
    dic: python dictionary containing all variables stats per year
    key: variable name
    year: year to extract
    Returns:
    --------
    indexes: these are the variable types (e.g. barque or nan)
    series.values: these are the counts of how many that variable name gets repeated
    year: year to sample
    """
    series = dic[key]
    indexes = series.index.values
    year = np.repeat(year, len(indexes))
    return indexes, series.values, year

In [ ]:
def exptract_year_arrays(path_to_file, key):
    """
    Reads pickle file and extracts the variable arrays per year
    Parms:
    -----
    path_to_file: path to the pickle file
    key: variable to extract
    Returns:
    --------
    df: dataframe from get_df

    """
    with open(path_to_file, "rb") as handle:
        base = os.path.basename(path_to_file)
        year = os.path.splitext(base)[0]
        dic_pickle = pickle.load(handle)
        df = get_values(dic_pickle, key, year)
        return df

In [ ]:
def make_data_frame(list_of_files, main_directory, key):
    # Define empty arrays to store the data
    years = np.array([])
    types_of_var = np.array([])
    counts_var = np.array([])

    for file in list_of_files:
        full_path = os.path.join(main_directory, file)
        var_type, count, year_f = exptract_year_arrays(full_path, key)
        years = np.concatenate([years, year_f])
        types_of_var = np.concatenate([types_of_var, var_type])
        counts_var = np.concatenate([counts_var, count])

    dataset = pd.DataFrame({"Year": years, key: types_of_var, "Count": counts_var})

    return dataset

In [ ]:
dirs

In [ ]:
# List of variables names stored in the pickle files
dic_keys = [
    "ship_types",
    "lan_inds",  # in a silly mistake I wrote lat wrong in the output data set. Oh well
    "lon_inds",
    "at_units",
    "sst_units",
    "ap_units",
    "bart_units",
    "lon_units",
    "baro_types",
    "distance_units",
    "distance_units_to_land",
    "distance_units_travelled",
    "units_of_other_measurements",
    "humidity_units",
]


df_ships = make_data_frame(file_names, dirs, dic_keys[0]).dropna()
df_lati = make_data_frame(file_names, dirs, dic_keys[1]).dropna()
df_loni = make_data_frame(file_names, dirs, dic_keys[2]).dropna()
df_atu = make_data_frame(file_names, dirs, dic_keys[3]).dropna()
df_sstu = make_data_frame(file_names, dirs, dic_keys[4]).dropna()
df_apu = make_data_frame(file_names, dirs, dic_keys[5]).dropna()
df_bartu = make_data_frame(file_names, dirs, dic_keys[6]).dropna()
df_lonu = make_data_frame(file_names, dirs, dic_keys[7]).dropna()
df_barot = make_data_frame(file_names, dirs, dic_keys[8]).dropna()

df_distu = make_data_frame(file_names, dirs, dic_keys[9]).dropna()
df_distu_land = make_data_frame(file_names, dirs, dic_keys[10]).dropna()
df_distu_travel = make_data_frame(file_names, dirs, dic_keys[11]).dropna()
df_unit_m = make_data_frame(file_names, dirs, dic_keys[12]).dropna()
df_humi_u = make_data_frame(file_names, dirs, dic_keys[13]).dropna()

- Ship types

In [ ]:
types_of_ships = df_ships.ship_types.unique()
types_of_ships

In [ ]:
len(types_of_ships)

Now we subdivide `ship_types` into groups that represent the types of sailing/steam ships or into a general category **sailing ship**; which covers all the different translations of the word ship in all languages of the data set.

In [ ]:
df = pd.DataFrame({"Types of Ship": types_of_ships})

Bark or Barque can also be refer as barc (e.g Falucho in [catalan](https://es.wikipedia.org/wiki/Barca_levantina))

In [ ]:
Bark_or_Barque = df[df["Types of Ship"].str.contains("|".join(["BARK", "FALUCHO"]))]
Bark_or_Barque

In [ ]:
Barkentine_or_Barquentine = df[
    df["Types of Ship"].str.contains("|".join(["BARQUEN", "BARKEN"]))
]
Barkentine_or_Barquentine

Brigantine

In [ ]:
Brigantine = df[
    df["Types of Ship"].str.contains(
        "|".join(["BRIG", "BRIGAN", "BRIK", "BERGANTIN", "BRICK", "BARGENTIJN"])
    )
]
Brigantine

In [ ]:
Schooner = df[df["Types of Ship"].str.contains("|".join(["SCHO", "GOLET"]))]
Schooner

In [ ]:
Frigate = df[df["Types of Ship"].str.contains("GAT", regex=False)]
Frigate

In [ ]:
Steam = df[df["Types of Ship"].str.contains("STEAM", regex=False)]
Steam

In [ ]:
Corvet = df[df["Types of Ship"].str.contains("|".join(["KORV", "CORVE"]))]
Corvet

In [ ]:
Cotter = df[df["Types of Ship"].str.contains("|".join(["KOTT", "COTT", "CUTT"]))]
Cotter

In [ ]:
Sloop = df[df["Types of Ship"].str.contains("|".join(["SLOOP", "SLOEP"]))]
Sloop

In [ ]:
Snow = df[df["Types of Ship"].str.contains("|".join(["SNOW", "SNA"]))]
Snow

In [ ]:
Naval_salining_ships = df[
    df["Types of Ship"].str.contains("|".join(["TH RATE", "AVISO", "RATE"]))
]
Naval_salining_ships

In [ ]:
East_Indianman = df[df["Types of Ship"].str.contains("SPIEGELRETOURSC")]
East_Indianman

In [ ]:
Scow = df[df["Types of Ship"].str.contains("GABARRE")]
Scow

In [ ]:
Fluyt = df[df["Types of Ship"].str.contains("FLU")]
Fluyt

In [ ]:
Ships_all_languages = df[
    df["Types of Ship"].str.contains(
        "|".join(
            [
                "SHIP",
                "PAQUE",
                "PAKKET",
                "BUQUE",
                "VES",
                "SCHIP",
                "NAV",
                "TRANSP",
                "EXPLOR",
                "CHAMBE",
                "BALANDRA",
                "PINK",
                "HOEKER",
                "POLACRA",
                "KOOP",
            ]
        )
    )
]
Ships_all_languages

In [ ]:
total = (
    len(Bark_or_Barque)
    + len(Barkentine_or_Barquentine)
    + len(Brigantine)
    + len(Schooner)
    + len(Frigate)
    + len(Steam)
    + len(Corvet)
    + len(Cotter)
    + len(Sloop)
    + len(Snow)
    + len(Ships_all_languages)
    + len(East_Indianman)
    + len(Scow)
    + len(Fluyt)
)
total

There are about 9 sub types that are hard to classify and more research is needed.

- AT units

In [ ]:
df_atu.at_units.unique()

- SST units

In [ ]:
df_sstu.sst_units.unique()

- Air pressure units

In [ ]:
df_apu.ap_units.unique()

The **DLS** unit format is believed to stand for Dutch Lines and the 12R or 10R means that were reduced to 12 or 10 Réaumur (like pressures reduced to 32F). This is still an ongoing discussion.

- Attached thermometer units

In [ ]:
df_bartu.bart_units.unique()

- Longitude Units

In [ ]:
df_lonu.lon_units.unique()

- Barometer types:

In [ ]:
df_barot.baro_types.unique()

- Other units for which there was no information

In [ ]:
df_distu_land.distance_units_to_land.unique()

In [ ]:
df_distu_travel.distance_units_travelled.unique()

In [ ]:
df_unit_m.units_of_other_measurements.unique()

In [ ]:
df_humi_u.humidity_units.unique()

An overview through time of some of the Barometer types and air pressure units

In [ ]:
fig = plt.figure(figsize=(20, 10))

ax = fig.add_subplot(111)

g = sns.histplot(
    data=df_barot,
    x="Year",
    hue="baro_types",
    multiple="stack",
    palette="deep",
    ax=ax,
    legend=True,
)
ax.grid(False)
plt.setp(g.get_xticklabels(), rotation=45)
plt.title("Barometer types", fontsize=16)

plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 10))

ax = fig.add_subplot(111)

cmap = sns.set_palette("rocket", n_colors=len(df_apu.ap_units.unique()))

g = sns.histplot(
    data=df_apu,
    x="Year",
    hue="ap_units",
    multiple="stack",
    palette=cmap,
    ax=ax,
    legend=True,
)
ax.grid(False)
plt.setp(g.get_xticklabels(), rotation=45)
plt.title("Air temp units", fontsize=16)

plt.show()

**Overview of the position quality**

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(20, 10))


g = sns.histplot(
    data=df_lati,
    x="Year",
    hue="lan_inds",
    multiple="stack",
    palette="deep",
    ax=ax[0],
    legend=True,
)
plt.setp(g.get_xticklabels(), rotation=45)
ax[0].grid(False)

xticks = ax[0].xaxis.get_major_ticks()
for i in range(len(xticks)):
    if i % 2 == 1:
        xticks[i].set_visible(False)

ax[0].set_title(
    "Coordinates indicator", fontdict={"fontsize": 16, "fontweight": "medium"}
)
ax[0].set_xlabel("")


p = sns.histplot(
    data=df_loni,
    x="Year",
    hue="lon_inds",
    multiple="stack",
    palette="colorblind",
    ax=ax[1],
    legend=True,
)
plt.setp(p.get_xticklabels(), rotation=45)
ax[1].grid(False)

xticks = ax[1].xaxis.get_major_ticks()
for i in range(len(xticks)):
    if i % 2 == 1:
        xticks[i].set_visible(False)

plt.show()

Code table for lat and lon indicators, according to this [information](https://stvno.github.io/page/cliwoc/):

```
{
	"1":"originates from dead reckoning",
	"2":"originates from true navigation (bearing/distance, celestial)",
	"3":"Interpolated manually",
	"4":"Interpolated",
	"5":"Inserted actual position (ports, islands, etc.)",
	"6":"Missing"
}
```

Is it worth using the coordinates from the supplemental metadata or should I use the imma.core lat and lon?